In [1]:
import cv2

In [2]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import Qt
import sys

In [3]:
import operator

In [4]:
import urllib.request

In [5]:
from pyagender import PyAgender

Using TensorFlow backend.


In [6]:
import numpy as np

In [7]:
import math

In [8]:
import face_recognition

In [9]:
from fer import FER

Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
agender = PyAgender()

21-01-2021:15:37:03,468 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



21-01-2021:15:37:03,471 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



21-01-2021:15:37:03,474 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.



21-01-2021:15:37:03,505 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



21-01-2021:15:37:03,507 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



...

21-01-2021:15:37:03,518 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.



21-01-2021:15:37:03,530 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



21-01-2021:15:37:03,541 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



21-01-2021:15:37:03,653 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



21-01-2021:15:37:04,333 WARNING  [module_wrapper.py:139] From C:\Users\UTVTRA\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [11]:
#face detection model
faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"
faceNet=cv2.dnn.readNet(faceModel,faceProto)

In [12]:
#gender detection
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
genderNet=cv2.dnn.readNet(genderModel,genderProto)
genderList=['Male','Female']
MODEL_MEAN_VALUES=(78.4263377603,87.7689143744,114.895847746)

In [13]:
#Harcascade face detection
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [14]:
def new_face(frame):
    new_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5, minSize=(30,30))
    frameWidth=new_frame.shape[1]
    
    face_list = []
    for (x, y, w, h) in faces:
        cv2.rectangle(new_frame, (x,y),(x+w, y+h),(40, 40, 40),2)
        face_list.append([x,y,x+w,y+h])

    
    return new_frame, face_list

In [15]:
#highlights the face and also returns the frame with placing the rectangles on the faces
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)
    

    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]

    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]

        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)

            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (40,40,40), int(round(frameHeight/150)), 1)
    
    #faceBoxes = face_recognition.face_locations(frameOpencvDnn)
    
    #for each_face in faceBoxes:
        
    

    return frameOpencvDnn,faceBoxes

In [16]:
video = cv2.VideoCapture(0)

In [17]:
'''def detectFace():
    while cv2.waitKey(1) < 0:
        padding = 20
        hasFrame,frame=video.read()
        resultImg, faceBoxes = highlightFace(faceNet,frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if(len(faceBoxes) != 0):

            for faceBox in faceBoxes:
                face = frame[max(0,faceBox[1]-padding):min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding):min(faceBox[2]+padding,frame.shape[1]-1)]
                blob = cv2.dnn.blobFromImage(face,1.0,(227,227),MODEL_MEAN_VALUES,swapRB=False)
                genderNet.setInput(blob)
                genderPreds = genderNet.forward()
                gender = genderList[genderPreds[0].argmax()]
                faces_agender = agender.detect_genders_ages(resultImg)
                print(faces_agender[0]['age'])
                try:
                    #expression = DeepFace.analyze(resultImg, actions = ['emotion'])
                    cv2.rectangle(resultImg,(faceBox[2]+5,faceBox[1]),(faceBox[2]+120,faceBox[1]+30),(128,128,128),-1)
                    cv2.putText(resultImg, 'Gender: '+gender,(faceBox[2]+7, faceBox[1]+10),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
                    #cv2.putText(resultImg, 'Emotion: '+max(expression['emotion'].items(), key=operator.itemgetter(1))[0],(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
                    cv2.putText(resultImg, 'Age: '+faces_agender[0]['age'],(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
                    #cv2.putText(resultImg,max(expression['emotion'].items(), key=operator.itemgetter(1))[0], (faceBox[0], faceBox[1]+15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1, cv2.LINE_AA)
                except Exception as e:
                    print('gender not found by model')
        cv2.imshow('faces', resultImg)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyWindow('faces')
            cv2.waitKey(1)
            video.release
            break '''

"def detectFace():\n    while cv2.waitKey(1) < 0:\n        padding = 20\n        hasFrame,frame=video.read()\n        resultImg, faceBoxes = highlightFace(faceNet,frame)\n        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n        \n        if(len(faceBoxes) != 0):\n\n            for faceBox in faceBoxes:\n                face = frame[max(0,faceBox[1]-padding):min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding):min(faceBox[2]+padding,frame.shape[1]-1)]\n                blob = cv2.dnn.blobFromImage(face,1.0,(227,227),MODEL_MEAN_VALUES,swapRB=False)\n                genderNet.setInput(blob)\n                genderPreds = genderNet.forward()\n                gender = genderList[genderPreds[0].argmax()]\n                faces_agender = agender.detect_genders_ages(resultImg)\n                print(faces_agender[0]['age'])\n                try:\n                    #expression = DeepFace.analyze(resultImg, actions = ['emotion'])\n                    cv2.rectangle(resultImg

In [18]:
#loading the images from the files
happy_img = cv2.imread('happy2.jpg')
sad_img = cv2.imread('sad2.jpg')
anger_img = cv2.imread('angry2.jpg')
neutral_img = cv2.imread('neutral2.jpg')
surprise_img = cv2.imread('surprise2.jpg')

#Method to resize the emojis with width 50 and height 50
happy = cv2.resize(happy_img,(50,50))
sad = cv2.resize(sad_img,(50,50))
anger = cv2.resize(anger_img,(50,50))
neutral = cv2.resize(neutral_img,(50,50))
surprise = cv2.resize(surprise_img,(50,50))

In [19]:
#detectFace()
detector = FER()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


21-01-2021:15:37:21,683 WARNING  [deprecation.py:506] From C:\Users\UTVTRA\anaconda3\envs\newface\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [36]:
#model testing
sarath_img = cv2.imread('sarath.jfif')
faces_sarath = agender.detect_genders_ages(sarath_img)
emo = detector.top_emotion(sarath_img)

In [37]:
#face location with new face recognition package
face_location = face_recognition.face_locations(sarath_img)
face_location

[(32, 135, 94, 73)]

In [38]:
faces_sarath

[{'left': 41,
  'top': 0,
  'right': 170,
  'bottom': 126,
  'width': 129,
  'height': 126,
  'gender': 0.113611184,
  'age': 31.368002505971162}]

In [39]:
#Py-Agender testing
faces_sarath

[{'left': 41,
  'top': 0,
  'right': 170,
  'bottom': 126,
  'width': 129,
  'height': 126,
  'gender': 0.113611184,
  'age': 31.368002505971162}]

In [23]:
#emotion-testing
emo

('happy', 0.53)

In [24]:
def make_1080():
    video.set(3,1920)
    video.set(4,1080)

In [25]:
def make_720():
    video.set(3,1280)
    video.set(4,720)

In [26]:
def make_TV():
    video.set(3,1440)
    video.set(4,2560)

In [27]:
def make_special():
    video.set(3, 2200)
    video.set(4, 1080)

In [28]:
make_1080()

In [27]:
make_TV()

In [28]:
make_special()

In [34]:
while cv2.waitKey(1) < 0:
    hasFrame,frame = video.read()
    resultImg, faceBoxes = highlightFace(faceNet,frame)
    #resultImg, faceBoxes = new_face(frame)
    print(faceBoxes)
    for faceBox in faceBoxes:
        padding = (faceBox[3]-faceBox[1])*0.25
        face = frame[math.ceil(max(0,faceBox[1]-padding)):math.ceil(min(faceBox[3]+padding,frame.shape[0]-1)),math.ceil(max(0,faceBox[0]-padding)):math.ceil(min(faceBox[2]+padding,frame.shape[1]-1))]
        
        try:
            blob = cv2.dnn.blobFromImage(face,1.0,(227,227),MODEL_MEAN_VALUES,swapRB=False)
            genderNet.setInput(blob)

            genderPreds = genderNet.forward()
            gender = genderList[genderPreds[0].argmax()]
            faces_agender = agender.detect_genders_ages(face)
            emotion, score = detector.top_emotion(face)
            
            print(faces_agender)
            cv2.rectangle(resultImg,(faceBox[2]+5,faceBox[1]),(faceBox[2]+70,faceBox[1]+30),(50,255,255),-1)
            #cv2.rectangle(frame,(faces_agender[0]['left'],faces_agender[0]['top']),(faces_agender[0]['right'],faces_agender[0]['bottom']),int(round(faces_agender[0]['bottom']-faces_agender[0]['top']/150)),-1)
            cv2.putText(resultImg, gender+', '+str(math.floor(faces_agender[0]['age'])),(faceBox[2]+7, faceBox[1]+10),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            #cv2.putText(resultImg, 'Age: '+str(math.floor(faces_agender[0]['age'])),(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            
            
            background_img = resultImg
            
            if(emotion == "happy"):
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, happy[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg, "Happy",(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            elif(emotion == "neutral"): #neutral
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, neutral[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg, "neutral",(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            elif(emotion == "sad" or emotion == "disgust" or emotion == "fear"): #sad
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, sad[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg, "sad",(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            elif(emotion == "angry"): #angry
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, anger[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg,"angry",(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            elif(emotion == "surprise"): #angry
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, anger[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                v2.putText(resultImg,"surprise",(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            cv2.imshow('detect age',background_img)
    
        except Exception as e:
            print('exception thrown')
            cv2.namedWindow('detect age',cv2.WINDOW_NORMAL)
            #cv2.setWindowProperty('detect age',cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            cv2.imshow('detect age',resultImg)
            
    cv2.namedWindow('detect age',cv2.WINDOW_NORMAL)
    #cv2.setWindowProperty('detect age',cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)        
    cv2.imshow('detect age',resultImg)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow('detect age')
        cv2.waitKey(1)
        video.release
        break

[[894, 327, 1254, 801]]
[]
exception thrown
[[903, 331, 1272, 791]]
[]
exception thrown
[[899, 329, 1269, 799]]
[]
exception thrown
[[900, 329, 1266, 807]]
[{'left': 0, 'top': 5, 'right': 604, 'bottom': 716, 'width': 604, 'height': 711, 'gender': 0.026248481, 'age': 30.172522560561447}]
[[888, 324, 1256, 817]]
exception thrown
[[890, 325, 1257, 815]]
exception thrown
[[894, 323, 1261, 808]]
exception thrown
[[893, 322, 1259, 807]]
exception thrown
[[881, 324, 1249, 805]]
exception thrown
[[880, 321, 1248, 809]]
exception thrown
[[875, 319, 1248, 807]]
[]
exception thrown
[[876, 316, 1239, 787]]
[{'left': 0, 'top': 0, 'right': 597, 'bottom': 705, 'width': 597, 'height': 705, 'gender': 0.013909101, 'age': 28.402071922210325}]
[[866, 306, 1231, 773]]
[]
exception thrown
[[870, 308, 1231, 787]]
[{'left': 0, 'top': 0, 'right': 599, 'bottom': 717, 'width': 599, 'height': 717, 'gender': 0.014541059, 'age': 29.50153840938492}]
[[866, 309, 1224, 778]]
[{'left': 0, 'top': 0, 'right': 592, 'botto

In [90]:
#[[654 343 317 317]].tolist()

In [32]:
while cv2.waitKey(1) < 0:
    hasFrame,frame = video.read()
    #resultImg, faceBoxes = highlightFace(faceNet,frame)
    resultImg, faceBoxes = new_face(frame)
    print(faceBoxes)
    for faceBox in faceBoxes:
        padding = (faceBox[3]-faceBox[1])*0
        face = frame[math.ceil(max(0,faceBox[1]-padding)):math.ceil(min(faceBox[3]+padding,frame.shape[0]-1)),math.ceil(max(0,faceBox[0]-padding)):math.ceil(min(faceBox[2]+padding,frame.shape[1]-1))]
        
        try:
            cv2.namedWindow('',flags=cv2.WINDOW_GUI_NORMAL)
            blob = cv2.dnn.blobFromImage(face,1.0,(227,227),MODEL_MEAN_VALUES,swapRB=False)
            genderNet.setInput(blob)

            genderPreds = genderNet.forward()
            gender = genderList[genderPreds[0].argmax()]
            faces_agender = agender.detect_genders_ages(face)
            emotion, score = detector.top_emotion(face)
            
            print(faces_agender)
            cv2.rectangle(resultImg,(faceBox[2]+5,faceBox[1]),(faceBox[2]+140,faceBox[1]+70),(50,255,255),-1)
            #cv2.rectangle(frame,(faces_agender[0]['left'],faces_agender[0]['top']),(faces_agender[0]['right'],faces_agender[0]['bottom']),int(round(faces_agender[0]['bottom']-faces_agender[0]['top']/150)),-1)
            cv2.putText(resultImg, gender+', '+str(math.floor(faces_agender[0]['age'])),(faceBox[2]+7, faceBox[1]+22),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            #cv2.putText(resultImg, 'Age: '+str(math.floor(faces_agender[0]['age'])),(faceBox[2]+7, faceBox[1]+25),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,0), 1, cv2.LINE_AA)
            
            
            background_img = resultImg
            
            if(emotion == "happy"):
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, happy[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg, "Happy",(faceBox[2]+7, faceBox[1]+50),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            elif(emotion == "neutral"): #neutral
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, neutral[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg, "neutral",(faceBox[2]+7, faceBox[1]+50),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            elif(emotion == "sad" or emotion == "disgust" or emotion == "fear"): #sad
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, sad[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg, "sad",(faceBox[2]+7, faceBox[1]+50),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            elif(emotion == "angry"): #angry
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, anger[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg,"angry",(faceBox[2]+7, faceBox[1]+50),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            elif(emotion == "surprise"): #angry
                mix_img = cv2.addWeighted(background_img[faceBox[1]-60:faceBox[1]-10, (int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25,:],0.3, anger[0:100,0:100,:],0.7,0)
                background_img[faceBox[1]-60:faceBox[1]-10,(int((faceBox[0]+faceBox[2])/2))-25:(int((faceBox[0]+faceBox[2])/2))+25] = mix_img
                cv2.putText(resultImg,"angry",(faceBox[2]+7, faceBox[1]+50),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2, cv2.LINE_AA)
            cv2.imshow('',background_img)
    
        except Exception as e:
            print('exception thrown')
            #cv2.namedWindow('',cv2.WINDOW_NORMAL)
            #cv2.setWindowProperty('detect age',cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            cv2.namedWindow('',flags = cv2.WINDOW_GUI_EXPANDED)
            cv2.imshow('',resultImg)
            
            
    
    #cv2.setWindowProperty('detect age',cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)        
    cv2.namedWindow('',flags = cv2.WINDOW_GUI_EXPANDED)
    cv2.imshow('',resultImg)
    
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyWindow('')
        cv2.waitKey(1)
        video.release
        break

[[887, 461, 1282, 856]]
[{'left': 0, 'top': 0, 'right': 394, 'bottom': 394, 'width': 394, 'height': 394, 'gender': 0.18944247, 'age': 33.936352607634035}]
[[814, 432, 1241, 859]]
[{'left': 0, 'top': 0, 'right': 426, 'bottom': 426, 'width': 426, 'height': 426, 'gender': 0.048286777, 'age': 30.502220653299446}]
[[832, 452, 1243, 863]]
[{'left': 0, 'top': 0, 'right': 410, 'bottom': 410, 'width': 410, 'height': 410, 'gender': 0.046186417, 'age': 32.011799362284364}]
[[847, 461, 1236, 850]]
[{'left': 0, 'top': 0, 'right': 388, 'bottom': 388, 'width': 388, 'height': 388, 'gender': 0.07206063, 'age': 33.41393216892175}]
[[826, 455, 1223, 852]]
[{'left': 0, 'top': 0, 'right': 396, 'bottom': 396, 'width': 396, 'height': 396, 'gender': 0.06394355, 'age': 28.870249679395783}]
[[816, 451, 1224, 859]]
[{'left': 0, 'top': 0, 'right': 407, 'bottom': 407, 'width': 407, 'height': 407, 'gender': 0.065296724, 'age': 27.897979507250966}]
[[835, 463, 1206, 834]]
[{'left': 0, 'top': 0, 'right': 370, 'bottom

In [41]:
21 in [21,22,45]

True